## GIL (Global Interpreter Lock)
*파이썬의 아킬레스건*

- CPython이 단일 바이트코드를 수행할 때
- Thread가 interrupt하지 못하도록 전역 인터프리터 잠금을 수행합니다.
- Thread가 여러개 돌고 있는 상황에서 인터프리터가 올바르게 동작하도록

<span style='color:blue'>**그 결과.**</span>
- <u>Multithreading 상황에서도 GIL은 한 번에 한 스레드에서만 동작하게 합니다.</u>
- Multithreading의 본래의 장점이 잘 발휘되지 않습니다.
- Multiprocess는 물론 영향을 받지 않습니다. 다만 *multithread보다는 다루기 어렵습니다.*

In [1]:
def factorize(number):
    return [i for i in range(1, number+1) if number % i == 0]

import time

numbers = [2139079, 1214759, 1516637, 1852285] * 4 # 값을 바꿔서 heavy하게 돌려보세요.
start = time.time()
for number in numbers:
    factorize(number)
end = time.time()
print('Took %.3f seconds' % (end-start))

Took 1.828 seconds


In [2]:
from concurrent.futures import ThreadPoolExecutor

pool = ThreadPoolExecutor(max_workers=4)
start = time.time()
results = list(pool.map(factorize, numbers))
end = time.time()

print('Took %.3f seconds' % (end-start))

Took 2.217 seconds


In [3]:
from concurrent.futures import ProcessPoolExecutor

pool = ProcessPoolExecutor(max_workers=4)
start = time.time()
results = list(pool.map(factorize, numbers))
end = time.time()

print('Took %.3f seconds' % (end-start))

Took 0.576 seconds


## 만약 IO를 다룬다면?
![](http://az616578.vo.msecnd.net/files/2016/02/20/635915954198393408-286853401_What-If-Analysis.jpg)

In [4]:
import select
import time
def slow_systemcall():
    select.select([], [], [], 0.1)

start = time.time()
for _ in range(5):
    slow_systemcall()
end = time.time()

print('Took %.3f seconds' % (end-start))

Took 0.507 seconds


In [5]:
from concurrent.futures import ThreadPoolExecutor

start = time.time()
with ThreadPoolExecutor(max_workers=4) as e:
    for _ in range(5):
        e.submit(slow_systemcall)
end = time.time()

print('Took %.3f seconds' % (end-start))

Took 0.209 seconds


In [6]:
from concurrent.futures import ProcessPoolExecutor

start = time.time()
with ProcessPoolExecutor(max_workers=4) as e:
    for _ in range(5):
        e.submit(slow_systemcall)
end = time.time()

print('Took %.3f seconds' % (end-start))

Took 0.223 seconds


IO에 의한 대기는 GIL이 영향을 받지 않으므로, IO부하가 큰 작업은 Thread로도 충분합니다. 상황에 맞게 Concurrency 방법을 선택하세요. 시스템 프로그래머가 아니면 thread를 heavy하게 쓸 일은 거의 없고, 위에 소개된 concurrent.futures는 쉽고 강력한 cuncurrency를 구현할 수 있습니다.